In [1]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
import csv

from math import radians, sin, cos, sqrt, atan2
import random


## Failure: GPT prompt engineering

In [2]:
# Load environment variables from .env file
load_dotenv()

# Access the API key using the variable name defined in the .env file
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")
print(client.api_key)

sk-proj-4WN8zdQt5xC1ajBjhvDoT3BlbkFJ6TlarliMdtBQ268Snowp


In [3]:
def read_tourism_spots(csv_file_path):
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_string = file.read()
    return csv_string

# CSV 파일 경로 설정
csv_file_path = './data/small_set.csv'

kr_locations = read_tourism_spots(csv_file_path)

In [4]:
print(kr_locations)

POI_ID,POI_NM,CTPRVN_NM,SIGNGU_NM,LC_LO,LC_LA
7231419,임랑해수욕장,부산광역시,기장군,129.2642611,35.31869433
7232777,오이도해양단지,경기도,시흥시,126.6902931,37.34640271
16758706,구로시장떡볶이골목,서울특별시,구로구,126.8855426,37.48881811
20868088,덕천동젊음의거리,부산광역시,북구,129.0070903,35.20989557
21085594,광안리카페거리,부산광역시,수영구,129.1163874,35.15091579
21085658,동대문패션타운,서울특별시,중구,127.0086616,37.56866403
21085686,을지로타일상가,서울특별시,중구,126.98898,37.56637497
21085883,사강회센터,경기도,화성시,126.7362366,37.21304632
21086097,까치울음식테마마을,경기도,부천시,126.8138061,37.51514031
10592447,신숭겸장군유적지,대구광역시,동구,128.6416639,35.94253672
10593745,목포근대역사관,전라남도,목포시,126.382103,34.78754765


In [5]:
# prompt engineering depends on the system's content
system_cont_ls = ["You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.", 
"You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.\nPlease provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).",
"You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.\nPlease provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).\nPlease organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO).",
"You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.\nPlease provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).\nPlease organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO).\nPlease provide the name of your area in English (Korean). The basic number of travel days is 2 days.",
"You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.\nPlease provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).\nPlease organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO).\nPlease provide the name of your area in English (Korean). The basic number of travel days is 2 days.\nIf the maximum length is insufficient, please include only POI_NM, CTPRVN_NM by day.",
"You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.\nPlease provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).\nPlease organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO).\nPlease provide the name of your area in English (Korean). The basic number of travel days is 2 days.\nIf the maximum length is insufficient, please include only POI_NM, CTPRVN_NM by day.\nDon't include the explain."]

In [6]:
assistant_answer_ls = []
# GPT-3.5 Turbo API를 사용하여 여행 계획을 생성하는 함수
def generate_travel_plan(tourism_spots, sys_cont):

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=1,
        max_tokens=512,
        top_p=0.7,
        frequency_penalty=0,
        presence_penalty=0,
        messages=[
            { "role": "system", "content": """See Korea tourist attraction:\n{}. {}""".format(kr_locations, sys_cont)},
            { "role": "user",  "content": "Make a plan for a trip to the sea in Korea"},
        ],
        stream=False,
    )
    # 응답에서 생성된 계획 추출
    generated_plan = completion.choices[0].message

    return generated_plan

for i in range(len(system_cont_ls)):
    travel_plan = generate_travel_plan(kr_locations, system_cont_ls[i]).content
    assistant_answer_ls.append(travel_plan) # save the anwers
    # print(i+1, "answer:\n")
    # print(travel_plan)

In [7]:
j=0
for i in assistant_answer_ls:
    print(f'system {j+1}:  \n{system_cont_ls[j]}\n')
    print(f'answer {j+1}:  \n{i}')
    j+=1
    print("\n------------------------------------------------------------------------------------------------\n")

system 1:  
You are the best travel planner who can make efficient travel plans.
Please write based on the data I presented.

answer 1:  
For a relaxing trip to the sea in Korea, I recommend visiting the following locations:

1. **Gwangalli Beach (광안리해수욕장)** in Busan
   - Enjoy the sandy beach and vibrant atmosphere.
   - Explore the nearby Gwangalli Cafe Street for some delicious coffee and snacks.

2. **Oido Marine Complex (오이도해양단지)** in Gyeonggi Province
   - Experience the beauty of the ocean and enjoy water activities like boating and fishing.
   - Take a stroll along the promenade and enjoy the sea breeze.

3. **Kkachisan Food Theme Village (까치울음식테마마을)** in Gyeonggi Province
   - Indulge in a variety of delicious local foods and experience the unique culinary culture of the region.
   - Explore the village's charming streets and shops.

4. **Limnang Beach (임랑해수욕장)** in Busan
   - Relax on the sandy beach and soak up the sun.
   - Take a refreshing dip in the clear waters of the s

In [12]:
import markdown2

# Markdown 형식의 텍스트를 HTML로 변환하여 파일에 저장
with open("travel_plans.html", "w", encoding="utf-8") as file:
    file.write("<html>\n<head>\n<title>Travel Plans</title>\n</head>\n<body>\n")
    j = 0
    for i in assistant_answer_ls:
        html_content = markdown2.markdown(i)
        file.write(f"<h2>Answer {j+1}</h2>\n{html_content}\n<hr>\n")
        j += 1
    file.write("</body>\n</html>")

위도 경도: LC_LO,LC_LA
POI_NM: 무슨 거리/골목

## 냐옹

In [ ]:
df = pd.read_csv('./data/small_set.csv')

In [ ]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # 지구의 반지름 (단위: km)
    R = 6371.0

    # 위도 및 경도를 라디안으로 변환
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Haversine 공식 적용
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

def calculate_distances(locations):
    distances = []

    # 관광지의 개수만큼 반복하여 거리를 계산합니다.
    for i in range(len(locations) - 1):
        lat1, lon1 = locations.iloc[i]['LC_LA'], locations.iloc[i]['LC_LO']
        lat2, lon2 = locations.iloc[i + 1]['LC_LA'], locations.iloc[i + 1]['LC_LO']

        # 거리 계산 함수 호출
        distance = calculate_distance(lat1, lon1, lat2, lon2)
        distances.append(distance)

    return distances

# 최적의 동선을 찾는 함수
def find_optimal_route(locations):
    # 모든 가능한 순열을 생성하고 가장 짧은 거리를 가진 순열을 찾습니다.
    from itertools import permutations
    
    min_distance = float('inf')
    optimal_route = None

    # 순열 생성
    for perm in permutations(locations.index):
        perm_locations = locations.loc[list(perm)]
        distances = calculate_distances(perm_locations)
        total_distance = sum(distances)

        # 현재 순열의 거리가 최소 거리보다 작으면 갱신합니다.
        if total_distance < min_distance:
            min_distance = total_distance
            optimal_route = perm

    return optimal_route, min_distance

# 선택된 관광지 출력
# 선택된 관광지 출력
def print_selected_locations(selected_locations):
    for idx, row in selected_locations.iterrows():
        print(f"POI_ID: {idx}, POI_NM: {row['POI_NM']}, CTPRVN_NM: {row['CTPRVN_NM']}")

selected_locations = df.sample(n=6)
# 최적의 동선 찾기
optimal_route, min_distance = find_optimal_route(selected_locations)

# 최적의 동선에 따른 관광지 출력
print("최적의 동선에 따른 관광지 출력:")
print_selected_locations(selected_locations)

# 결과 출력
print("최단 이동 거리:", min_distance)


# optimal_route, min_distance = find_optimal_route(selected_locations)



# # 결과 출력
# print("최적의 동선:", optimal_route)
# print("최단 이동 거리:", min_distance)
